# Amazon SageMaker LightGBM Bring Your Own Model
_**Hosting a pre-trained LightGBM model in an Amazon SageMaker scikit-learn Container**_

---

## Background

Amazon SageMaker includes functionality to support a hosted notebook environment, distributed, serverless training, and real-time hosting. We think it works best when all three of these services are used together, but they can also be used independently.  Some use cases may only require hosting.  Maybe the model was trained prior to Amazon SageMaker existing, in a different service.

We show how to use a pre-trained LightGBM model with the Amazon SageMaker scikit-learn container, including how to specify dependencies. Then, we register that model with the Amazon SageMaker model registry, which allows for version tracking. Finally, we use the registered model to transform batch of data, as might be run from AWS Lambda, and also to create a real-time hosted endpoint.

We use the California Housing dataset, present in Scikit-Learn: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html. That dataset was originally published in:

> Pace, R. Kelley, and Ronald Barry. "Sparse spatial auto-regressions." Statistics & Probability Letters 33.3 (1997): 291-297.

---
## Setup

Ensure we have the latest version of the SageMaker Python SDK. Also install LightGBM.

In [ ]:
import sys
import subprocess

subprocess.check_call(
    [sys.executable, "-m", "pip", "install", "-U", "sagemaker", "lightgbm"]
)

Let's start by specifying:

* AWS region.
* The IAM role arn used to give learning and hosting access to your data.
* The S3 bucket that you want to use for training and model data.

In [ ]:
import os
import time
import re
import json
import pandas as pd
import numpy as np
import tarfile
import sagemaker
import boto3
from sagemaker import get_execution_role, image_uris, ModelPackage
from sagemaker.sklearn.model import SKLearnModel, SKLearnPredictor
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

region = boto3.Session().region_name

role = get_execution_role()

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-sklearn-byo-model"

print(f"bucket: {bucket}")
print(f"sagemaker version: {sagemaker.__version__}")

## Prepare data for model inference

We load the California housing dataset from sklearn, and will use it to invoke SageMaker Endpoint

In [ ]:
data = fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

# we don't train a model, so we will need only the testing data
testX = pd.DataFrame(X_test, columns=data.feature_names)

testX.head(10)

## Create the pre-trained model file

In [ ]:
import lightgbm as lgb
import joblib

model = lgb.LGBMRegressor()

model.fit(X_train, y_train)

print(f"Predicted:\t{model.predict(X_test[:5, :])}")
print(f"Actual:\t\t{y_test[:5]}")

model_file_name = "model.joblib"

joblib.dump(model, model_file_name)

## Write the Inference Script

When using endpoints with the Amazon SageMaker managed `Scikit Learn` container, we need to provide an entry point script for inference that will **at least** load the saved model.

After the SageMaker model server has loaded your model by calling `model_fn`, SageMaker will serve your model. Model serving is the process of responding to inference requests, received by SageMaker `InvokeEndpoint` API calls.


We will implement also the `predict_fn()` function that takes the deserialized request object and performs inference against the loaded model.

We will now create this script and call it `inference.py` and store it at the root of a directory called `code`.

**Note:** You would modify the script below to implement your own inferencing logic.

Additional information on model loading and model serving for scikit-learn on SageMaker can be found in the [SageMaker Scikit-learn Model Server documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#deploy-a-scikit-learn-model)

There are also several functions for hosting which we won't define,
 - `input_fn()` - Takes request data and deserializes the data into an object for prediction.
 - `output_fn()` - Takes the result of prediction and serializes this according to the response content type.

These will take on their default values as described [SageMaker Scikit-learn Serve a Model documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html#serve-a-model)

In [ ]:
model_code_path = "./code"
model_code_inference = "inference.py"

os.makedirs(model_code_path, exist_ok=True)

In [ ]:
%%writefile $model_code_path/$model_code_inference

import os
import joblib


def predict_fn(input_object, model):
    ###########################################
    # Do your custom preprocessing logic here #
    ###########################################

    print("calling model")
    predictions = model.predict(input_object)
    return predictions


def model_fn(model_dir):
    print("loading model.joblib from: {}".format(model_dir))
    loaded_model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return loaded_model

### Installing additional Python dependencies

It also may be necessary to supply a `requirements.txt` file to ensure any necessary dependencies are installed in the container along with the script. For this script, in addition to the Python standard libraries, we showcase how to install the `boto3` and `requests` libraries.

In [ ]:
%%writefile $model_code_path/requirements.txt

lightgbm

## Package the pre-trained model in `model.tar.gz` and upload it to S3
The model file name must satisfy the regular expression pattern: `^[a-zA-Z0-9](-*[a-zA-Z0-9])*;`, and needs to be tar-zipped.

In [ ]:
model_tar_name = "model.tar.gz"
with tarfile.open(model_tar_name, "w:gz") as tar:
    tar.add(model_file_name)

Upload the model to S3.

In [ ]:
key = os.path.join(prefix, model_tar_name)
s3 = boto3.client("s3")
s3.upload_file(model_tar_name, bucket, key)
model_data = f"s3://{bucket}/{key}"
print(f"model data: {model_data}")

### Create the model

Here we showcase the process of creating a model from s3 artifacts, that could be used to deploy a model that was trained in a different session or even out of SageMaker.

In [ ]:
model = SKLearnModel(
    role=role,
    model_data=model_data,
    framework_version="0.23-1",
    py_version="py3",
    source_dir=model_code_path,
    entry_point=model_code_inference,
    sagemaker_session=sagemaker.Session(),  # Required for model.register().
)

### Register the model version
Create a model group.

In [ ]:
sm_client = boto3.client("sagemaker", region_name=region)
model_package_group_name = "scikit-housing-prediction" + str(round(time.time()))
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "For predicting ln(median house value)",
}

create_model_pacakge_group_response = sm_client.create_model_package_group(
    **model_package_group_input_dict
)
model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
print("ModelPackageGroup Arn : {}".format(model_package_group_arn))

Register the model to the model group.

In [ ]:
model_package = model.register(
    content_types=["text/csv", "application/json"],
    response_types=["text/csv", "application/json"],
    inference_instances=["ml.t2.medium"],
    transform_instances=["ml.m5.large"],
    model_package_group_name=model_package_group_name,
    description="Predict house values",
    approval_status="Approved",
)
model_package_arn = model_package.model_package_arn
print(model_package_arn)

### Batch transform data.

Upload the test data to S3.

In [ ]:
np.savetxt("X_test.csv", X_test, delimiter=",")

bucket = sagemaker.Session().default_bucket()
X_test_prefix = f"{prefix}/input/X_test.csv"
s3.upload_file("X_test.csv", bucket, X_test_prefix)
X_test_S3 = f"s3://{bucket}/{X_test_prefix}"
output_path = f"s3://{bucket}/{prefix}/output"

print(X_test_S3)
print(output_path)

Create a transformer and process the data.

In [ ]:
model_from_package = ModelPackage(
    role=role,
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker.Session(),
)

transformer = model_from_package.transformer(
    instance_count=1, instance_type="ml.m5.large", output_path=output_path
)

Processing the data will take 5-10 minutes to complete.

In [ ]:
%%time

transformer.transform(X_test_S3, content_type="text/csv", split_type="Line")
transformer.wait()

Download the batch transform results and print them.

In [ ]:
s3.download_file(bucket, f"{prefix}/output/X_test.csv.out", "X_test.csv.out")

with open("X_test.csv.out", "r") as f:
    X_test_out = json.load(f)

print(f"Predicted:\t{X_test_out[:5]}")
print(f"Actual:\t\t{y_test[:5]}")

### Create a real-time endpoint
Create an endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 5-10 minutes to complete.

In [ ]:
%%time

endpoint_name = f"scikit-housing-prediction-{str(round(time.time()))}"
model_from_package.deploy(
    instance_type="ml.t2.medium", initial_instance_count=1, endpoint_name=endpoint_name
)
predictor = SKLearnPredictor(endpoint_name=endpoint_name)

Let's generate the prediction for the test data generated earlier.

In [ ]:
# the SKLearnPredictor does the serialization from pandas for us
predictions = predictor.predict(testX[data.feature_names])

print(f"Predicted:\t{predictions[:5]}")
print(f"Actual:\t\t{y_test[:5]}")

If you're ready to be done with this endpoint, please run the delete_endpoint line in the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
predictor.delete_endpoint()